In [1]:
!pip install pycocotools
!pip install rouge
!pip install gdown
!pip install pycocoevalcap
!pip install utils

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.3/104.3 MB 9.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for utils: filename=utils-1.0.2-py2.py3-none-any.whl size=13906 sha256=614f7c9fa1b34283f5cf0235d54b01283424aad545e74e2a79317712c2dfc903
  Stored in directory: /root/.cache/pip/wheels/15/0c/b3/674aea8c5d91c642c817d4d630bd58faa316724b136844094d
Successfully built utils


In [ ]:
import os
import sys
import time
import json
import random
import numpy as np
from typing import Dict, List, Tuple, Optional, Union, Any
import pickle
from collections import defaultdict
import glob
import re
import itertools
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import torchvision
import torchvision.transforms as transforms
from torchvision.models import resnet101, ResNet101_Weights
import matplotlib.pyplot as plt
import nltk
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction
from nltk.translate.meteor_score import meteor_score
from rouge import Rouge
from pycocoevalcap.cider.cider import Cider
from pycocoevalcap.spice.spice import Spice
import skimage.transform
from PIL import Image
import nltk
import os
import sys
import time
import json
import random
import numpy as np
from typing import Dict, List, Tuple, Optional, Union, Any
import pickle
from collections import defaultdict
import glob
import re
import itertools
from tqdm.notebook import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

import torchvision
import torchvision.transforms as transforms
from torchvision.models import resnet101, ResNet101_Weights

import matplotlib.pyplot as plt
import nltk
from nltk.translate.bleu_score import corpus_bleu
from nltk.translate.meteor_score import meteor_score
from nltk.translate.bleu_score import SmoothingFunction
from rouge import Rouge



# Download NLTK resources
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('punkt_tab')

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True

# Check for GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt_tab to /root/nltk_data...


Using device: cuda


[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [3]:
def download_and_extract_coco():
    """
    Download MS COCO dataset and extract it
    """
    # URL for official COCO annotations
    annotation_url = "http://images.cocodataset.org/annotations/annotations_trainval2014.zip"
    train_images_url = "http://images.cocodataset.org/zips/train2014.zip"
    val_images_url = "http://images.cocodataset.org/zips/val2014.zip"

    # Create directories
    os.makedirs('data/coco/annotations', exist_ok=True)
    os.makedirs('data/coco/images', exist_ok=True)

    # Download and extract annotations
    if not os.path.exists('data/coco/annotations/captions_train2014.json'):
        print("Downloading annotation zip file...")
        !wget {annotation_url} -P data/coco/annotations/
        print("Extracting annotations...")
        !unzip -q data/coco/annotations/annotations_trainval2014.zip -d data/coco/annotations/
        !rm data/coco/annotations/annotations_trainval2014.zip

    # Download and extract train images
    if not os.path.exists('data/coco/images/train2014'):
        print("Downloading train images (this may take a while)...")
        !wget {train_images_url} -P data/coco/images/
        print("Extracting train images...")
        !unzip -q data/coco/images/train2014.zip -d data/coco/images/
        !rm data/coco/images/train2014.zip

    # Download and extract validation images
    if not os.path.exists('data/coco/images/val2014'):
        print("Downloading validation images (this may take a while)...")
        !wget {val_images_url} -P data/coco/images/
        print("Extracting validation images...")
        !unzip -q data/coco/images/val2014.zip -d data/coco/images/
        !rm data/coco/images/val2014.zip

    print("Dataset download and extraction complete!")

In [5]:
def create_vocab_and_tokenize(annotation_file: str, threshold: int = 5) -> Tuple[Dict, Dict]:
    """
    Create word-to-index and index-to-word mappings and tokenize captions

    Args:
        annotation_file: Path to the training annotation file (e.g., captions_train2014.json)
        threshold: Minimum word count threshold

    Returns:
        word2idx: Dictionary mapping words to indices
        idx2word: Dictionary mapping indices to words
    """
    # Load training annotations
    with open(annotation_file, 'r') as f:
        data = json.load(f)

    # Extract all captions from annotations
    annotations = data['annotations']
    captions = [ann['caption'] for ann in annotations]

    # Count word frequency
    word_counts = defaultdict(int)
    for caption in captions:
        tokens = nltk.tokenize.word_tokenize(caption.lower())
        for token in tokens:
            word_counts[token] += 1

    # Create vocabulary (only keep words that appear more than threshold times)
    words = [w for w, c in word_counts.items() if c >= threshold]

    # Create mappings
    word2idx = {w: i + 4 for i, w in enumerate(words)}  # 0: <pad>, 1: <start>, 2: <end>, 3: <unk>
    word2idx['<pad>'] = 0
    word2idx['<start>'] = 1
    word2idx['<end>'] = 2
    word2idx['<unk>'] = 3

    idx2word = {w: i for i, w in word2idx.items()}

    print(f"Vocabulary size: {len(word2idx)}")

    return word2idx, idx2word

In [ ]:
class COCODataset(Dataset):
    """COCO Custom Dataset compatible with torch.utils.data.DataLoader"""

    def __init__(self,
                 root: str,
                 annotation_file: str,
                 split: str,
                 vocab: Dict,
                 transform=None,
                 use_bottomup: bool = True,
                 bottomup_features_dir: Optional[str] = None,
                 max_seq_length: int = 16):
        """
        Args:
            root: Image directory
            annotation_file: Path to annotation file (e.g., captions_train2014.json or captions_val2014.json)
            split: 'train' or 'val'
            vocab: Word-to-index dictionary
            transform: Image transformations
            use_bottomup: Whether to use Bottom-Up features
            bottomup_features_dir: Directory containing Bottom-Up features
            max_seq_length: Maximum sequence length
        """
        self.root = root
        self.split = split  # 'train' or 'val'
        self.vocab = vocab
        self.transform = transform
        self.max_seq_length = max_seq_length
        self.use_bottomup = use_bottomup
        self.bottomup_features_dir = bottomup_features_dir

        # Load annotations
        with open(annotation_file, 'r') as f:
            self.data = json.load(f)

        # Create mapping from image_id to captions
        image_id_to_captions = defaultdict(list)
        for ann in self.data['annotations']:
            image_id_to_captions[ann['image_id']].append(ann['caption'])

        # Process images
        self.images = []
        for img in self.data['images']:
            img_info = {
                'id': img['id'],  # Use 'id' instead of 'cocoid'
                'filename': img['file_name'],
                'filepath': 'train2014' if split == 'train' else 'val2014',
                'sentences': [{'raw': cap} for cap in image_id_to_captions[img['id']]]
            }
            self.images.append(img_info)

        if self.use_bottomup:
            # Load Bottom-Up features
            import h5py
            if split == 'train':
                self.features_file = h5py.File(os.path.join(bottomup_features_dir, 'train36.hdf5'), 'r')
            else:  # val
                self.features_file = h5py.File(os.path.join(bottomup_features_dir, 'val36.hdf5'), 'r')

            # Map image_id to features index
            self.features_idx = {}
            with open(os.path.join(bottomup_features_dir, f"{split}36_imgid2idx.pkl"), 'rb') as f:
                self.features_idx = pickle.load(f)

    def __len__(self) -> int:
        return len(self.images)

    def __getitem__(self, idx: int) -> Tuple:
        """
        Returns:
            image_features: Image features (CNN features or Bottom-Up features)
            caption: Caption sequence (tokenized and numericalized)
            caption_length: Caption length
            all_captions: All captions for the image (for evaluation)
            image_id: Image ID
        """
        image_info = self.images[idx]
        image_id = image_info['id']  # Changed from 'cocoid' to 'id'

        # Get image features
        if self.use_bottomup:
            try:
                features_idx = self.features_idx[image_id]
                image_features = torch.from_numpy(self.features_file['features'][features_idx])
                spatial_features = torch.from_numpy(self.features_file['boxes'][features_idx])
                image_features = torch.cat([image_features, spatial_features], dim=1)
            except KeyError:
                # If Bottom-Up features are not available, use a zero tensor
                image_features = torch.zeros(36, 2048 + 4)  # 36 regions, 2048 features + 4 spatial features
        else:
            # Load and transform image
            image_path = os.path.join(self.root, image_info['filepath'], image_info['filename'])
            image = Image.open(image_path).convert('RGB')
            if self.transform is not None:
                image = self.transform(image)
            image_features = image

        # Randomly select a caption
        caption_idx = random.randint(0, len(image_info['sentences']) - 1)
        caption = image_info['sentences'][caption_idx]

        # Tokenize and numericalize caption
        tokens = nltk.tokenize.word_tokenize(caption['raw'].lower())
        caption = []
        caption.append(self.vocab['<start>'])
        caption.extend([self.vocab.get(token, self.vocab['<unk>']) for token in tokens])
        caption.append(self.vocab['<end>'])

        # Cap caption length
        caption_length = len(caption)
        if caption_length > self.max_seq_length:
            caption = caption[:self.max_seq_length]
            caption[-1] = self.vocab['<end>']
            caption_length = self.max_seq_length

        # Get all captions for evaluation
        all_captions = []
        for sent in image_info['sentences']:
            tokens = nltk.tokenize.word_tokenize(sent['raw'].lower())
            all_captions.append(tokens)

        return image_features, torch.tensor(caption), caption_length, all_captions, image_id


def collate_fn(batch):
    """
    Custom collate function for DataLoader to handle variable length sequences

    Args:
        batch: Batch of samples from Dataset

    Returns:
        images: Batch of image features
        captions: Batch of padded caption sequences
        lengths: Caption lengths
        all_captions: All captions for each image
        image_ids: Image IDs
    """
    # Sort batch by caption length (descending)
    batch.sort(key=lambda x: x[2], reverse=True)

    # Separate data
    images, captions, lengths, all_captions, image_ids = zip(*batch)

    # Stack images
    images = torch.stack(images, 0) if not isinstance(images[0], torch.Tensor) or images[0].dim() < 3 else torch.cat([img.unsqueeze(0) for img in images], 0)

    # Pad captions
    max_length = max(lengths)
    padded_captions = torch.zeros(len(captions), max_length).long()
    for i, cap in enumerate(captions):
        end = lengths[i]
        padded_captions[i, :end] = cap[:end]

    return images, padded_captions, torch.tensor(lengths), all_captions, image_ids


In [7]:

class ImageEncoder(nn.Module):
    """
    Image Encoder using ResNet101
    """
    def __init__(self, embed_dim: int = 2048, train_cnn: bool = False):
        """
        Args:
            embed_dim: Feature dimension
            train_cnn: Whether to train the CNN
        """
        super(ImageEncoder, self).__init__()

        # Load pre-trained ResNet101
        resnet = resnet101(weights=ResNet101_Weights.IMAGENET1K_V1)
        modules = list(resnet.children())[:-1]  # Remove the last FC layer
        self.resnet = nn.Sequential(*modules)

        # Linear layer to project features to embed_dim
        self.linear = nn.Linear(resnet.fc.in_features, embed_dim)
        self.bn = nn.BatchNorm1d(embed_dim)
        self.dropout = nn.Dropout(0.5)

        # Freeze ResNet if not training
        if not train_cnn:
            for param in self.resnet.parameters():
                param.requires_grad = False

    def forward(self, images: torch.Tensor) -> torch.Tensor:
        """
        Extract image features

        Args:
            images: Images of shape (batch_size, 3, height, width)

        Returns:
            features: Image features of shape (batch_size, embed_dim)
        """
        with torch.set_grad_enabled(self.training):
            features = self.resnet(images)

        features = features.reshape(features.size(0), -1)
        features = self.linear(features)
        features = self.bn(features)
        features = self.dropout(features)

        return features

class BottomUpEncoder(nn.Module):
    """
    Bottom-Up attention features encoder
    """
    def __init__(self, input_dim: int = 2048 + 4, embed_dim: int = 2048):
        """
        Args:
            input_dim: Input feature dimension (2048 + 4 for Bottom-Up features)
            embed_dim: Output embedding dimension
        """
        super(BottomUpEncoder, self).__init__()

        # Linear layer to project features to embed_dim
        self.linear = nn.Linear(input_dim, embed_dim)
        self.bn = nn.BatchNorm1d(embed_dim)
        self.dropout = nn.Dropout(0.5)

    def forward(self, features: torch.Tensor) -> torch.Tensor:
        """
        Project Bottom-Up features

        Args:
            features: Bottom-Up features of shape (batch_size, num_regions, input_dim)

        Returns:
            projected_features: Projected features of shape (batch_size, num_regions, embed_dim)
        """
        batch_size, num_regions, _ = features.size()

        # Reshape for batch normalization
        features = features.reshape(batch_size * num_regions, -1)

        # Project features
        projected_features = self.linear(features)
        projected_features = self.bn(projected_features)
        projected_features = self.dropout(projected_features)

        # Reshape back
        projected_features = projected_features.reshape(batch_size, num_regions, -1)

        return projected_features

In [8]:

class Attention(nn.Module):
    """
    Attention module for the decoder
    """
    def __init__(self, encoder_dim: int, decoder_dim: int, attention_dim: int):
        """
        Args:
            encoder_dim: Feature dimension of encoded images
            decoder_dim: Dimension of decoder's hidden state
            attention_dim: Dimension of attention network
        """
        super(Attention, self).__init__()

        self.encoder_att = nn.Linear(encoder_dim, attention_dim)
        self.decoder_att = nn.Linear(decoder_dim, attention_dim)
        self.full_att = nn.Linear(attention_dim, 1)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)

    def forward(self, encoder_out: torch.Tensor, decoder_hidden: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:
        """
        Compute attention weights and context vector

        Args:
            encoder_out: Encoded images, shape (batch_size, num_pixels/regions, encoder_dim)
            decoder_hidden: Decoder hidden state, shape (batch_size, decoder_dim)

        Returns:
            context: Context vector, shape (batch_size, encoder_dim)
            alpha: Attention weights, shape (batch_size, num_pixels/regions)
        """
        # Projection
        encoder_att = self.encoder_att(encoder_out)  # (batch_size, num_pixels/regions, attention_dim)
        decoder_att = self.decoder_att(decoder_hidden).unsqueeze(1)  # (batch_size, 1, attention_dim)

        # Calculate attention scores
        att = self.relu(encoder_att + decoder_att)  # (batch_size, num_pixels/regions, attention_dim)
        att = self.full_att(att).squeeze(2)  # (batch_size, num_pixels/regions)

        # Softmax to get attention weights
        alpha = self.softmax(att)  # (batch_size, num_pixels/regions)

        # Compute context vector
        context = (encoder_out * alpha.unsqueeze(2)).sum(dim=1)  # (batch_size, encoder_dim)

        return context, alpha

class DecoderWithAttention(nn.Module):
    """
    Decoder with Attention for image captioning
    """
    def __init__(self,
                 embed_dim: int,
                 decoder_dim: int,
                 vocab_size: int,
                 encoder_dim: int = 2048,
                 attention_dim: int = 512,
                 dropout: float = 0.5):
        """
        Args:
            embed_dim: Embedding dimension
            decoder_dim: Dimension of decoder's hidden state
            vocab_size: Size of vocabulary
            encoder_dim: Feature dimension of encoded images
            attention_dim: Dimension of attention network
            dropout: Dropout rate
        """
        super(DecoderWithAttention, self).__init__()

        self.encoder_dim = encoder_dim
        self.attention_dim = attention_dim
        self.embed_dim = embed_dim
        self.decoder_dim = decoder_dim
        self.vocab_size = vocab_size
        self.dropout = dropout

        # Attention network
        self.attention = Attention(encoder_dim, decoder_dim, attention_dim)

        # Embedding layer
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.dropout = nn.Dropout(dropout)

        # LSTM decoder
        self.lstm = nn.LSTMCell(embed_dim + encoder_dim, decoder_dim)

        # Linear layer for initial hidden and cell states
        self.init_h = nn.Linear(encoder_dim, decoder_dim)
        self.init_c = nn.Linear(encoder_dim, decoder_dim)

        # Linear layer to find scores over vocabulary
        self.fc = nn.Linear(decoder_dim, vocab_size)

        # Initialize weights
        self.init_weights()

    def init_weights(self):
        """
        Initialize weights
        """
        self.embedding.weight.data.uniform_(-0.1, 0.1)
        self.fc.bias.data.fill_(0)
        self.fc.weight.data.uniform_(-0.1, 0.1)

    def init_hidden_state(self, encoder_out: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:
        """
        Initialize hidden state and cell state

        Args:
            encoder_out: Encoded images, shape (batch_size, num_pixels/regions, encoder_dim)

        Returns:
            h, c: Initial hidden state and cell state, shape (batch_size, decoder_dim)
        """
        mean_encoder_out = encoder_out.mean(dim=1)  # (batch_size, encoder_dim)
        h = self.init_h(mean_encoder_out)  # (batch_size, decoder_dim)
        c = self.init_c(mean_encoder_out)  # (batch_size, decoder_dim)
        return h, c

    def forward(self,
               encoder_out: torch.Tensor,
               encoded_captions: torch.Tensor,
               caption_lengths: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
        """
        Forward pass

        Args:
            encoder_out: Encoded images, shape (batch_size, num_pixels/regions, encoder_dim)
            encoded_captions: Encoded captions, shape (batch_size, max_caption_length)
            caption_lengths: Caption lengths, shape (batch_size, 1)

        Returns:
            predictions: Prediction scores, shape (batch_size, max_caption_length, vocab_size)
            alphas: Attention weights, shape (batch_size, max_caption_length, num_pixels/regions)
        """
        batch_size = encoder_out.size(0)
        num_pixels = encoder_out.size(1)

        # Sort input by caption length (descending)
        caption_lengths, sort_ind = caption_lengths.sort(descending=True)
        encoder_out = encoder_out[sort_ind]
        encoded_captions = encoded_captions[sort_ind]

        # Embedding
        embeddings = self.embedding(encoded_captions)  # (batch_size, max_caption_length, embed_dim)

        # Initialize LSTM state
        h, c = self.init_hidden_state(encoder_out)  # (batch_size, decoder_dim)

        # Create tensors to hold prediction outputs and attention weights
        max_length = max(caption_lengths)
        predictions = torch.zeros(batch_size, max_length, self.vocab_size).to(device)
        alphas = torch.zeros(batch_size, max_length, num_pixels).to(device)

        # At each time-step, decode and compute attention
        for t in range(max_length):
            # Compute attention
            context, alpha = self.attention(encoder_out, h)

            # Store attention weights
            alphas[:, t, :] = alpha

            # LSTM input for this time step
            lstm_input = torch.cat([embeddings[:, t, :], context], dim=1)

            # LSTM step
            h, c = self.lstm(lstm_input, (h, c))

            # Generate prediction
            preds = self.fc(self.dropout(h))
            predictions[:, t, :] = preds

            # Break if all captions are processed
            if t >= caption_lengths[0]:
                break

        return predictions, alphas, encoded_captions, caption_lengths, sort_ind

    def sample(self, encoder_out: torch.Tensor, beam_size: int = 5) -> List[List[int]]:
        """
        Generate captions using beam search

        Args:
            encoder_out: Encoded images, shape (1, num_pixels/regions, encoder_dim)
            beam_size: Beam size

        Returns:
            seq: Generated sequence (most probable)
        """
        k = beam_size
        vocab_size = self.vocab_size

        # Initialize beam search
        encoder_out = encoder_out.expand(k, encoder_out.size(1), encoder_out.size(2))  # (k, num_pixels/regions, encoder_dim)

        # Initialize hidden state and cell state
        h, c = self.init_hidden_state(encoder_out)  # (k, decoder_dim)

        # Start with '<start>' token
        start_token = torch.LongTensor([1] * k).to(device)  # (k,)
        embeddings = self.embedding(start_token)  # (k, embed_dim)

        # Initialize sequences, scores, and completed sequences
        seqs = start_token.unsqueeze(1)  # (k, 1)
        top_k_scores = torch.zeros(k, 1).to(device)  # (k, 1)
        complete_seqs = []
        complete_seqs_scores = []

        # Beam search
        step = 1
        while True:
            # Compute attention
            context, _ = self.attention(encoder_out, h)

            # LSTM input
            lstm_input = torch.cat([embeddings, context], dim=1)

            # LSTM step
            h, c = self.lstm(lstm_input, (h, c))

            # Generate prediction
            output = self.fc(h)  # (k, vocab_size)

            # Add log probabilities
            scores = top_k_scores.expand_as(output) + F.log_softmax(output, dim=1)  # (k, vocab_size)

            # For the first step, keep only the first beam
            if step == 1:
                scores = scores[0].unsqueeze(0)  # (1, vocab_size)

            # Get top k words and their scores
            top_k_scores, top_k_words = scores.view(-1).topk(k, 0, True, True)  # (k,)

            # Convert to beam indices and word indices
            prev_beam_indices = top_k_words // vocab_size  # (k,)
            next_word_indices = top_k_words % vocab_size  # (k,)

            # Build new sequences
            new_seqs = torch.cat([seqs[prev_beam_indices], next_word_indices.unsqueeze(1)], dim=1)  # (k, step+1)

            # Check if any sequences are complete
            incomplete_indices = []
            for i, word_idx in enumerate(next_word_indices):
                if word_idx == 2:  # <end> token
                    complete_seqs.append(new_seqs[i].tolist())
                    complete_seqs_scores.append(top_k_scores[i].item())
                else:
                    incomplete_indices.append(i)

            # Stop if all beams are complete
            k = len(incomplete_indices)
            if k == 0:
                break

            # Update sequences, scores, and LSTM states
            seqs = new_seqs[incomplete_indices]
            top_k_scores = top_k_scores[incomplete_indices].unsqueeze(1)
            h = h[prev_beam_indices[incomplete_indices]]
            c = c[prev_beam_indices[incomplete_indices]]
            encoder_out = encoder_out[prev_beam_indices[incomplete_indices]]

            # Prepare input for next step
            embeddings = self.embedding(next_word_indices[incomplete_indices])

            # Increment step
            step += 1

            # Stop if maximum length reached
            if step > 50:
                break

        # Choose the sequence with the highest score
        i = complete_seqs_scores.index(max(complete_seqs_scores)) if complete_seqs_scores else 0
        seq = complete_seqs[i] if complete_seqs else seqs[0].tolist()

        return seq
class TopDownAttention(nn.Module):
    """
    Simplified Top-Down Attention for global image features (ResNet output)
    """
    def __init__(self,
                 embed_dim: int,
                 decoder_dim: int,
                 vocab_size: int,
                 word2idx: dict,  # Added parameter to accept word2idx
                 encoder_dim: int = 512,  # Must match ResNet's embed_dim
                 dropout: float = 0.5):
        super().__init__()

        self.word2idx = word2idx  # Store word2idx as an instance attribute
        self.encoder_dim = encoder_dim
        self.embed_dim = embed_dim
        self.decoder_dim = decoder_dim
        self.vocab_size = vocab_size
        self.dropout = dropout

        # Embedding layer
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.dropout_layer = nn.Dropout(dropout)

        # Simplified LSTM (no region attention)
        self.attention_lstm = nn.LSTMCell(encoder_dim + embed_dim, decoder_dim)
        self.language_lstm = nn.LSTMCell(decoder_dim, decoder_dim)

        # Output layer
        self.fc = nn.Linear(decoder_dim, vocab_size)

        # Initialize weights
        self.init_weights()

    def init_weights(self):
        self.embedding.weight.data.uniform_(-0.1, 0.1)
        self.fc.bias.data.fill_(0)
        self.fc.weight.data.uniform_(-0.1, 0.1)

    def forward(self, encoder_out, encoded_captions, caption_lengths):
        """
        Args:
            encoder_out: (batch_size, encoder_dim) - ResNet global features
            encoded_captions: (batch_size, max_caption_length)
            caption_lengths: (batch_size,)
        """
        batch_size = encoder_out.size(0)

        # Sort by caption length (descending)
        caption_lengths, sort_ind = caption_lengths.sort(dim=0, descending=True)
        encoder_out = encoder_out[sort_ind]
        encoded_captions = encoded_captions[sort_ind]

        # Embed captions
        embeddings = self.embedding(encoded_captions)  # (batch_size, max_len, embed_dim)

        # Initialize LSTM states
        h_att = torch.zeros(batch_size, self.decoder_dim).to(encoder_out.device)
        c_att = torch.zeros(batch_size, self.decoder_dim).to(encoder_out.device)
        h_lang = torch.zeros(batch_size, self.decoder_dim).to(encoder_out.device)
        c_lang = torch.zeros(batch_size, self.decoder_dim).to(encoder_out.device)

        # Output tensors
        max_len = max(caption_lengths)
        predictions = torch.zeros(batch_size, max_len, self.vocab_size).to(encoder_out.device)

        for t in range(max_len):
            # Simplified input (no region features)
            att_lstm_input = torch.cat([embeddings[:, t, :], encoder_out], dim=1)
            h_att, c_att = self.attention_lstm(att_lstm_input, (h_att, c_att))

            # Language LSTM
            h_lang, c_lang = self.language_lstm(h_att, (h_lang, c_lang))

            # Prediction
            preds = self.fc(self.dropout_layer(h_lang))
            predictions[:, t, :] = preds

            # Early stop if all captions are processed
            if t >= caption_lengths[0]:
                break

        return predictions, None, encoded_captions, caption_lengths, sort_ind

    def sample(self, encoder_out, beam_size=5):
        """
        Generate a caption using beam search.

        Args:
            encoder_out: Encoded image features (1, encoder_dim)
            beam_size: Number of beams for beam search

        Returns:
            best_sequence: List of token indices representing the generated caption
        """
        k = beam_size
        vocab_size = self.vocab_size
        device = encoder_out.device

        # Expand for beam search
        encoder_out = encoder_out.unsqueeze(1).expand(-1, k, -1).reshape(-1, self.encoder_dim)

        # Initialize states
        h_att = torch.zeros(k, self.decoder_dim).to(device)
        c_att = torch.zeros(k, self.decoder_dim).to(device)
        h_lang = torch.zeros(k, self.decoder_dim).to(device)
        c_lang = torch.zeros(k, self.decoder_dim).to(device)

        # Start tokens
        start_tokens = torch.LongTensor([1] * k).to(device)  # Assuming 1 is <start> token
        embeddings = self.embedding(start_tokens)

        # Beam search setup
        seqs = start_tokens.unsqueeze(1)  # (k, 1)
        top_k_scores = torch.zeros(k, 1).to(device)
        complete_seqs = []
        complete_seqs_scores = []

        # Track active sequences
        active_seqs = list(range(k))

        # Beam search loop
        for step in range(50):  # Max caption length
            # Simplified LSTM step
            lstm_input = torch.cat([embeddings, encoder_out], dim=1)
            h_att, c_att = self.attention_lstm(lstm_input, (h_att, c_att))
            h_lang, c_lang = self.language_lstm(h_att, (h_lang, c_lang))

            # Predict next word
            scores = self.fc(h_lang)  # (k, vocab_size)
            scores = F.log_softmax(scores, dim=1)  # (k, vocab_size)

            # Add current scores to cumulative scores
            scores = top_k_scores.expand_as(scores) + scores  # (k, vocab_size)

            # Flatten scores for top-k selection
            if step == 0:
                scores = scores[0]  # First step: only consider first sequence
                top_k_scores, top_k_words = scores.topk(k, dim=0)
                top_k_scores = top_k_scores.unsqueeze(1)  # (k, 1)
                prev_word_inds = torch.zeros(k, dtype=torch.long).to(device)  # (k,)
                next_word_inds = top_k_words  # (k,)
            else:
                scores = scores.view(-1)  # (k * vocab_size)
                top_k_scores, top_k_pos = scores.topk(k, dim=0)  # (k,), (k,)
                prev_word_inds = top_k_pos // vocab_size  # (k,)
                next_word_inds = top_k_pos % vocab_size  # (k,)
                top_k_scores = top_k_scores.unsqueeze(1)  # (k, 1)

            # Update sequences
            new_seqs = torch.zeros(k, seqs.size(1) + 1, dtype=torch.long).to(device)
            new_h_att = torch.zeros_like(h_att)
            new_c_att = torch.zeros_like(c_att)
            new_h_lang = torch.zeros_like(h_lang)
            new_c_lang = torch.zeros_like(c_lang)
            new_active_seqs = []

            for i in range(k):
                prev_idx = prev_word_inds[i].item()
                next_word = next_word_inds[i].item()

                # Check if sequence is active
                if prev_idx not in active_seqs:
                    continue

                # Handle completed sequences
                if next_word == self.word2idx['<end>']:  # Now works with self.word2idx
                    complete_seqs.append(seqs[prev_idx].cpu().tolist())
                    complete_seqs_scores.append(top_k_scores[i].item())
                    continue

                # Update sequence
                new_seqs[i, :-1] = seqs[prev_idx]
                new_seqs[i, -1] = next_word
                new_h_att[i] = h_att[prev_idx]
                new_c_att[i] = c_att[prev_idx]
                new_h_lang[i] = h_lang[prev_idx]
                new_c_lang[i] = c_lang[prev_idx]
                new_active_seqs.append(i)

            # Update states and sequences
            seqs = new_seqs
            h_att = new_h_att
            c_att = new_c_att
            h_lang = new_h_lang
            c_lang = new_c_lang
            active_seqs = new_active_seqs

            # Update embeddings for next step
            embeddings = self.embedding(next_word_inds)

            # Early stopping
            if len(active_seqs) == 0:
                break

        # Select best sequence
        if complete_seqs:
            best_idx = complete_seqs_scores.index(max(complete_seqs_scores))
            best_sequence = complete_seqs[best_idx]
        else:
            # If no sequences completed, take the highest-scoring active sequence
            best_idx = top_k_scores.argmax().item()
            best_sequence = seqs[best_idx].cpu().tolist()

        return best_sequence

In [ ]:

class CNNDiscriminator(nn.Module):
    def __init__(self, embed_dim=512, encoder_dim=2048, vocab_size=10000, hidden_dim=512):
        super(CNNDiscriminator, self).__init__()

        # Image feature projection
        self.img_proj = nn.Linear(encoder_dim, hidden_dim) # Expects encoder_dim (e.g. 512)

        # Caption embedding
        self.embedding = nn.Embedding(vocab_size, embed_dim)

        # CNN layers with multiple kernel sizes
        self.conv_layers = nn.ModuleList([
            nn.Sequential(
                nn.Conv1d(embed_dim, hidden_dim // 4, kernel_size=k, padding=k // 2), # Changed padding here
                nn.BatchNorm1d(hidden_dim // 4),
                nn.ReLU()
            ) for k in [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 15, 16]
        ])

        # Highway layer
        # Input to highway is num_kernels * (hidden_dim // 4)
        # len(self.conv_layers) = 12. So, 12 * hidden_dim // 4 = 3 * hidden_dim
        self.highway_transform = nn.Linear(3 * hidden_dim, 3 * hidden_dim)
        self.highway_gate = nn.Linear(3 * hidden_dim, 3 * hidden_dim)

        # Fusion and classification
        # Combined features: 3 * hidden_dim (caption) + hidden_dim (image) = 4 * hidden_dim
        self.fusion = nn.Sequential(
            nn.Linear(4 * hidden_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(hidden_dim, 1),
            nn.Sigmoid()
        )

    def forward(self, image_features_input, captions, caption_lengths):
        """
        Args:
            image_features_input: Image features, shape (batch_size, encoder_dim) or (batch_size, num_regions, encoder_dim)
            captions: Captions, shape (batch_size, caption_length)
            caption_lengths: Caption lengths, shape (batch_size,)
        """
        # Process image features
        if image_features_input.dim() == 3: # (batch_size, num_regions, encoder_dim)
            img_f = image_features_input.mean(dim=1)  # (batch_size, encoder_dim)
        elif image_features_input.dim() == 2: # (batch_size, encoder_dim)
            img_f = image_features_input
        else:
            raise ValueError(f"CNNDiscriminator: Unexpected image_features_input dimension: {image_features_input.dim()}. Expected 2 or 3.")

        img_features_projected = self.img_proj(img_f)  # (batch_size, hidden_dim)

        # Process captions
        embeds = self.embedding(captions).permute(0, 2, 1) # (batch_size, embed_dim, seq_len)

        conv_outputs_list = []
        for conv_layer in self.conv_layers:
            conv_out = conv_layer(embeds)
            # Max pooling over time
            pooled_out = F.max_pool1d(conv_out, kernel_size=conv_out.size(2)).squeeze(2) # (batch_size, hidden_dim // 4)
            conv_outputs_list.append(pooled_out)

        caption_cnn_features = torch.cat(conv_outputs_list, dim=1) # (batch_size, 3 * hidden_dim)

        # Highway layer
        transform_highway = self.highway_transform(caption_cnn_features)
        gate_highway = torch.sigmoid(self.highway_gate(caption_cnn_features))
        caption_features_highway = gate_highway * transform_highway + (1 - gate_highway) * caption_cnn_features

        # Fusion
        combined = torch.cat([caption_features_highway, img_features_projected], dim=1) # (batch_size, 4 * hidden_dim)
        outputs = self.fusion(combined)
        return outputs

class RNNDiscriminator(nn.Module):
    """
    RNN-based discriminator to classify caption-image pairs as real or fake
    """
    def __init__(self, embed_dim=512, encoder_dim=2048, vocab_size=10000, hidden_dim=512):
        super(RNNDiscriminator, self).__init__()

        # Caption embedding
        self.embedding = nn.Embedding(vocab_size, embed_dim)

        # LSTM with image features as initial input
        self.lstm = nn.LSTM(embed_dim, hidden_dim, batch_first=True)

        # Image feature projection to LSTM hidden state
        self.img_to_hidden = nn.Linear(encoder_dim, hidden_dim)
        self.img_to_cell = nn.Linear(encoder_dim, hidden_dim)

        # Fusion and classification
        self.fusion = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(hidden_dim, 1),
            nn.Sigmoid()
        )

    def forward(self, image_features_input, captions, caption_lengths):
        """
        Args:
            image_features_input: Image features, shape (batch_size, encoder_dim) or (batch_size, num_regions, encoder_dim)
            captions: Captions, shape (batch_size, caption_length)
            caption_lengths: Caption lengths, shape (batch_size,)
        """
        batch_size = image_features_input.size(0)
        # Process image features
        if image_features_input.dim() == 3: # (batch_size, num_regions, encoder_dim)
            img_f = image_features_input.mean(dim=1)
        elif image_features_input.dim() == 2: # (batch_size, encoder_dim)
            img_f = image_features_input
        else:
            raise ValueError(f"RNNDiscriminator: Unexpected image_features_input dimension: {image_features_input.dim()}. Expected 2 or 3.")

        h0 = self.img_to_hidden(img_f).unsqueeze(0) # (1, batch_size, hidden_dim)
        c0 = self.img_to_cell(img_f).unsqueeze(0) # (1, batch_size, hidden_dim)

        caption_embed = self.embedding(captions) # (batch_size, seq_len, embed_dim)

        # Ensure caption_lengths are on CPU and are Tensors for pack_padded_sequence
        # Also, filter out zero lengths if any, as pack_padded_sequence requires lengths > 0
        # However, the collate_fn should ensure lengths are valid.
        # If lengths can be zero (e.g. from empty sampled captions), they need to be handled.
        # For now, assume lengths are valid and on device.

        # Create a mask for valid lengths (lengths > 0)
        valid_indices = [i for i, length in enumerate(caption_lengths.cpu().tolist()) if length > 0]
        if not valid_indices: # All lengths are zero
            # Return a default output, e.g., zeros, or handle as per model design
            return torch.zeros(batch_size, 1, device=image_features_input.device).sigmoid()


        # Filter inputs based on valid_indices
        _captions_embed_filtered = caption_embed[valid_indices]
        _caption_lengths_filtered = caption_lengths[valid_indices].cpu() # Must be on CPU
        _h0_filtered = h0[:, valid_indices, :]
        _c0_filtered = c0[:, valid_indices, :]

        packed = pack_padded_sequence(_captions_embed_filtered, _caption_lengths_filtered, batch_first=True, enforce_sorted=False)
        _, (hidden, cell) = self.lstm(packed, (_h0_filtered, _c0_filtered)) # hidden is (1, num_valid, hidden_dim)

        caption_features_filtered = hidden.squeeze(0) # (num_valid, hidden_dim)

        # Initialize outputs for all batch items
        final_outputs = torch.zeros(batch_size, 1, device=image_features_input.device)

        # Apply fusion only to valid indices
        fused_outputs_filtered = self.fusion(caption_features_filtered) # (num_valid, 1)

        # Place results back into the original batch structure
        for i, original_idx in enumerate(valid_indices):
            final_outputs[original_idx] = fused_outputs_filtered[i]

        return final_outputs

In [ ]:

from pycocoevalcap.cider.cider import Cider

def compute_cider_score(generated_captions: List[List[str]], reference_captions: List[List[List[str]]]) -> List[float]:
    """
    Compute CIDEr-D scores for generated captions

    Args:
        generated_captions: List of generated captions, where each caption is a list of word tokens.
                            Example: [['a', 'cat', 'sat'], ['two', 'dogs', 'ran']]
        reference_captions: A list of lists of reference captions. Each image's references are a list of captions,
                            and each caption is a list of word tokens.
                            Example: [[['a', 'cat', 'was', 'sitting'], ['the', 'feline', 'sat']], [['two', 'dogs', 'were', 'running']]]
    Returns:
        scores: CIDEr-D scores for each caption, as a list of floats.
    """
    cider_scorer = Cider()

    # Format for pycocoevalcap
    # res: {image_id: [{'caption': "hyp_string"}]}
    res = {i: [{'caption': ' '.join(hyp_token_list)}] for i, hyp_token_list in enumerate(generated_captions)}

    # gts: {image_id: ["ref_string_1", "ref_string_2", ...]}
    gts = {i: [' '.join(ref_token_list) for ref_token_list in refs_for_image] for i, refs_for_image in enumerate(reference_captions)}

    # Handle cases where inputs might be empty to avoid errors in cider_scorer
    if not res or not gts or len(res) != len(gts):
        # print("Warning: CIDEr inputs (res or gts) are empty or mismatched. Returning zero scores.")
        return [0.0] * len(generated_captions)

    # Ensure that all captions are non-empty strings, as CIDEr might error on empty strings for gts
    for img_id in gts:
        gts[img_id] = [s if s.strip() else "empty" for s in gts[img_id]]
        if not gts[img_id]: # if all refs were empty and became "empty", ensure at least one
            gts[img_id] = ["empty"]
    for img_id in res:
        res[img_id][0]['caption'] = res[img_id][0]['caption'] if res[img_id][0]['caption'].strip() else "empty"


    try:
        # Compute CIDEr-D scores
        # compute_score returns (average_score, list_of_scores_for_each_image)
        _average_score, per_instance_scores = cider_scorer.compute_score(gts, res)
    except Exception as e:
        # print(f"Error during CIDEr computation: {e}")
        # print(f"Sample gts item: {list(gts.items())[0] if gts else 'gts is empty'}")
        # print(f"Sample res item: {list(res.items())[0] if res else 'res is empty'}")
        return [0.0] * len(generated_captions) # Return default scores on error


    if isinstance(per_instance_scores, np.ndarray):
        return per_instance_scores.tolist()
    elif isinstance(per_instance_scores, list):
        return per_instance_scores
    # Handle the case where per_instance_scores might be a single float if only one item was scored
    elif isinstance(per_instance_scores, float):
        return [per_instance_scores]
    else:
        # print(f"Warning: Unexpected type for per_instance_scores from CIDEr: {type(per_instance_scores)}. Returning zeros.")
        return [0.0] * len(generated_captions)

In [ ]:
def compute_rewards(generated_captions, reference_captions, discriminator, images, vocab, device):
    """
    Compute rewards for generated captions using discriminator and CIDEr-D

    Args:
        generated_captions: Generated captions, list of token sequences
        reference_captions: Reference captions, list of token sequences
        discriminator: Discriminator model
        images: Image features
        vocab: Vocabulary dictionary, expected to have 'idx2word' key
        device: Device (CPU or GPU)

    Returns:
        rewards: Rewards for each caption
    """
    batch_size = len(generated_captions)
    rewards = torch.zeros(batch_size, device=device)

    # Calculate discriminator rewards
    with torch.no_grad():
        # Ensure generated_captions is not empty before proceeding
        if not generated_captions:
            # Handle empty generated_captions case, e.g., return zero rewards or raise error
            return rewards # Or handle appropriately

        max_len = max([len(cap) for cap in generated_captions if cap]) # Added check for non-empty cap
        if max_len == 0 and generated_captions: # All captions are empty lists
             max_len = 1 # Avoid zero-dim tensor if all captions are empty but list is not

        padded_caps = torch.zeros(batch_size, max_len).long().to(device)
        cap_lengths = torch.zeros(batch_size).long().to(device)

        for i, cap in enumerate(generated_captions):
            if cap: # Ensure cap is not empty
                length = len(cap)
                padded_caps[i, :length] = torch.tensor(cap, device=device) # Ensure tensor is on the correct device
                cap_lengths[i] = length
            else:
                cap_lengths[i] = 0


        # Ensure images are correctly shaped for the discriminator if they come from encoder_out[sort_ind]
        # If images is (batch_size, encoder_dim), it's fine.
        # If discriminator expects (batch_size, num_regions, encoder_dim) and images is (batch_size, encoder_dim),
        # this might need adjustment or the discriminator should handle both.
        # Assuming discriminator can handle (batch_size, encoder_dim) from ImageEncoder.
        disc_rewards = discriminator(images, padded_caps, cap_lengths)

    # Calculate CIDEr-D rewards
    lang_rewards = torch.zeros(batch_size, device=device)

    # Convert token indices to words
    # Access idx2word using dictionary key lookup
    local_idx2word = vocab['idx2word']
    hypotheses = [[local_idx2word[idx.item() if isinstance(idx, torch.Tensor) else idx] for idx in cap if (idx.item() if isinstance(idx, torch.Tensor) else idx) not in [0, 1, 2]] for cap in generated_captions]
    references = reference_captions  # Already in word format

    # Compute CIDEr-D scores
    if hypotheses and references: # Ensure lists are not empty
        cider_scores = compute_cider_score(hypotheses, references)
        if cider_scores: # Ensure cider_scores is not empty
            lang_rewards = torch.tensor(cider_scores, device=device, dtype=torch.float32)
        else:
            lang_rewards = torch.zeros(batch_size, device=device, dtype=torch.float32) # Default if no scores
    else:
        lang_rewards = torch.zeros(batch_size, device=device, dtype=torch.float32)


    # Combine rewards (lambda = 0.3 as per paper)
    # Ensure disc_rewards and lang_rewards are compatible for addition
    # disc_rewards is (batch_size, 1), lang_rewards is (batch_size,)
    rewards = 0.3 * disc_rewards.squeeze() + 0.7 * lang_rewards.squeeze()


    return rewards

def policy_gradient_loss(decoder_outputs, rewards, generated_caps, baseline_rewards):
    """
    Compute policy gradient loss with SCST

    Args:
        decoder_outputs: Log probabilities from generator, shape (batch_size, max_len, vocab_size)
        rewards: Rewards for sampled captions, shape (batch_size,)
        generated_caps: Generated caption indices, shape (batch_size, max_len)
        baseline_rewards: Rewards from greedy decoding, shape (batch_size,)

    Returns:
        pg_loss: Policy gradient loss
    """
    batch_size, max_len, vocab_size = decoder_outputs.size()

    # Get log probabilities of generated words
    log_probs = torch.gather(decoder_outputs, 2, generated_caps.unsqueeze(2)).squeeze(2)

    # Mask out padded positions
    mask = (generated_caps != 0).float()

    # Compute advantage (reward - baseline)
    advantage = rewards - baseline_rewards  # (batch_size,)

    # Compute policy gradient loss
    pg_loss = -torch.sum(log_probs * mask * advantage.unsqueeze(1)) / torch.sum(mask)

    return pg_loss

In [12]:
class EnsembleDiscriminator(nn.Module):
    """
    Ensemble of CNN and RNN discriminators
    """
    def __init__(self, cnn_discriminator, rnn_discriminator):
        super(EnsembleDiscriminator, self).__init__()
        self.cnn_discriminator = cnn_discriminator
        self.rnn_discriminator = rnn_discriminator

    def forward(self, images, captions, caption_lengths):
        """
        Combine predictions from both discriminators
        """
        cnn_output = self.cnn_discriminator(images, captions, caption_lengths)
        rnn_output = self.rnn_discriminator(images, captions, caption_lengths)
        # Average the outputs
        return (cnn_output + rnn_output) / 2

In [13]:
def train_adversarial(train_loader, val_loader, encoder, generator, cnn_discriminator, rnn_discriminator,
                      word2idx, idx2word, device, learning_rate=3e-4, num_epochs=20,
                      lambda_gen=1.0, lambda_adv=0.1, checkpoint_path='./checkpoints'):
    """
    Adversarial training loop with pre-training, SCST, and mismatched pairs
    """
    os.makedirs(checkpoint_path, exist_ok=True)

    # Define optimizers
    optimizer_gen = optim.Adam(list(encoder.parameters()) + list(generator.parameters()), lr=learning_rate)
    optimizer_disc_cnn = optim.Adam(cnn_discriminator.parameters(), lr=learning_rate)
    optimizer_disc_rnn = optim.Adam(rnn_discriminator.parameters(), lr=learning_rate)

    # Define losses
    criterion_gen = nn.CrossEntropyLoss(ignore_index=0)
    criterion_disc = nn.BCELoss()

    # Create ensemble discriminator
    ensemble_discriminator = EnsembleDiscriminator(cnn_discriminator, rnn_discriminator)

    # Pre-train generator (MLE for 25 epochs)
    print("Pre-training generator with MLE...")
    for epoch in range(10): # Restored original pre-training epochs
        encoder.train()
        generator.train()
        gen_losses = []

        progress_bar_gen_pretrain = tqdm(train_loader, desc=f'Pre-train Gen Epoch {epoch+1}/25')
        for i_batch, (images, captions, lengths, _, _) in enumerate(progress_bar_gen_pretrain):
            # Removed DEBUG_MAX_BATCHES_PRETRAIN limit
            images, captions, lengths = images.to(device), captions.to(device), lengths.to(device)

            optimizer_gen.zero_grad()
            encoder_out = encoder(images)
            outputs, _, caps_sorted, lengths_sorted, _ = generator(encoder_out, captions, lengths)

            targets = caps_sorted[:, 1:].reshape(-1)
            outputs = outputs[:, :-1].reshape(-1, outputs.shape[2])
            gen_loss = criterion_gen(outputs, targets)

            gen_loss.backward()
            optimizer_gen.step()
            gen_losses.append(gen_loss.item())
            progress_bar_gen_pretrain.set_postfix({'GenLoss': f'{gen_loss.item():.4f}'})

        print(f'Pre-train Gen Epoch {epoch+1}/25, Gen Loss: {np.mean(gen_losses) if gen_losses else "N/A (no batches processed)":.4f}')

    # Pre-train discriminators (10 epochs)
    print("Pre-training discriminators...")
    for epoch in range(5): # Restored original pre-training epochs
        cnn_discriminator.train()
        rnn_discriminator.train()
        disc_cnn_losses = []
        disc_rnn_losses = []

        progress_bar_disc_pretrain = tqdm(train_loader, desc=f'Pre-train Disc Epoch {epoch+1}/10')
        for i_batch, batch in enumerate(progress_bar_disc_pretrain):
            # Removed DEBUG_MAX_BATCHES_PRETRAIN limit
            images, captions, lengths, _, _ = batch
            images, captions, lengths = images.to(device), captions.to(device), lengths.to(device)
            batch_size = images.size(0)

            with torch.no_grad():
              current_encoder_out = encoder(images)

            with torch.no_grad():
              sampled_ids = [generator.sample(current_encoder_out[j:j+1]) for j in range(batch_size)]
              max_len = max([len(ids) for ids in sampled_ids]) if sampled_ids else 0
              fake_captions = torch.zeros(batch_size, max_len).long().to(device)
              fake_lengths = torch.zeros(batch_size).long().to(device)
              for j, ids in enumerate(sampled_ids):
                  if ids:
                    fake_captions[j, :len(ids)] = torch.tensor(ids, device=device)
                    fake_lengths[j] = len(ids)
                  else:
                    fake_lengths[j] = 0

            mismatch_indices = torch.randperm(batch_size)
            mismatch_captions = captions[mismatch_indices]
            mismatch_lengths = lengths[mismatch_indices]

            real_labels = torch.ones(batch_size, 1).to(device)
            fake_labels = torch.zeros(batch_size, 1).to(device)

            optimizer_disc_cnn.zero_grad()
            real_outputs_cnn = cnn_discriminator(current_encoder_out, captions, lengths)
            fake_outputs_cnn = cnn_discriminator(current_encoder_out, fake_captions, fake_lengths)
            mismatch_outputs_cnn = cnn_discriminator(current_encoder_out, mismatch_captions, mismatch_lengths)
            disc_loss_cnn = (criterion_disc(real_outputs_cnn, real_labels) +
                            0.5 * criterion_disc(fake_outputs_cnn, fake_labels) +
                            0.5 * criterion_disc(mismatch_outputs_cnn, fake_labels))
            disc_loss_cnn.backward()
            optimizer_disc_cnn.step()
            disc_cnn_losses.append(disc_loss_cnn.item())

            optimizer_disc_rnn.zero_grad()
            real_outputs_rnn = rnn_discriminator(current_encoder_out, captions, lengths)
            fake_outputs_rnn = rnn_discriminator(current_encoder_out, fake_captions, fake_lengths)
            mismatch_outputs_rnn = rnn_discriminator(current_encoder_out, mismatch_captions, mismatch_lengths)
            disc_loss_rnn = (criterion_disc(real_outputs_rnn, real_labels) +
                            0.5 * criterion_disc(fake_outputs_rnn, fake_labels) +
                            0.5 * criterion_disc(mismatch_outputs_rnn, fake_labels))
            disc_loss_rnn.backward()
            optimizer_disc_rnn.step()
            disc_rnn_losses.append(disc_loss_rnn.item())
            progress_bar_disc_pretrain.set_postfix({
                'D_CNNLoss': f'{disc_loss_cnn.item():.4f}',
                'D_RNNLoss': f'{disc_loss_rnn.item():.4f}'
            })

        print(f'Pre-train Disc Epoch {epoch+1}/10, D_CNN Loss: {np.mean(disc_cnn_losses) if disc_cnn_losses else "N/A"}, D_RNN Loss: {np.mean(disc_rnn_losses) if disc_rnn_losses else "N/A"}')

    # Adversarial training
    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}/{num_epochs}')
        encoder.train()
        generator.train()
        cnn_discriminator.train()
        rnn_discriminator.train()

        gen_losses = []
        adv_losses = []
        disc_cnn_losses_main = []
        disc_rnn_losses_main = []

        progress_bar_main = tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}')
        for i_batch, (images, captions, lengths, all_captions, _) in enumerate(progress_bar_main):
            # Removed DEBUG_MAX_BATCHES_MAIN_EPOCH_1_2 limit
            images, captions, lengths = images.to(device), captions.to(device), lengths.to(device)
            batch_size = images.size(0)

            # --- Update Discriminators ---
            with torch.no_grad():
                encoder_out_disc = encoder(images)

                sampled_ids = [generator.sample(encoder_out_disc[j:j+1]) for j in range(batch_size)]
                max_len = max([len(ids) for ids in sampled_ids]) if sampled_ids else 0
                fake_captions = torch.zeros(batch_size, max_len).long().to(device)
                fake_lengths = torch.zeros(batch_size).long().to(device)
                for j, ids in enumerate(sampled_ids):
                    if ids:
                        fake_captions[j, :len(ids)] = torch.tensor(ids, device=device)
                        fake_lengths[j] = len(ids)
                    else:
                        fake_lengths[j] = 0

            mismatch_indices = torch.randperm(batch_size)
            mismatch_captions = captions[mismatch_indices]
            mismatch_lengths = lengths[mismatch_indices]

            real_labels = torch.ones(batch_size, 1).to(device)
            fake_labels = torch.zeros(batch_size, 1).to(device)

            optimizer_disc_cnn.zero_grad()
            real_outputs_cnn = cnn_discriminator(encoder_out_disc, captions, lengths)
            fake_outputs_cnn = cnn_discriminator(encoder_out_disc, fake_captions, fake_lengths)
            mismatch_outputs_cnn = cnn_discriminator(encoder_out_disc, mismatch_captions, mismatch_lengths)
            disc_loss_cnn_val = (criterion_disc(real_outputs_cnn, real_labels) +
                                0.5 * criterion_disc(fake_outputs_cnn, fake_labels) +
                                0.5 * criterion_disc(mismatch_outputs_cnn, fake_labels))
            disc_loss_cnn_val.backward()
            optimizer_disc_cnn.step()
            disc_cnn_losses_main.append(disc_loss_cnn_val.item())

            optimizer_disc_rnn.zero_grad()
            real_outputs_rnn = rnn_discriminator(encoder_out_disc, captions, lengths)
            fake_outputs_rnn = rnn_discriminator(encoder_out_disc, fake_captions, fake_lengths)
            mismatch_outputs_rnn = rnn_discriminator(encoder_out_disc, mismatch_captions, mismatch_lengths)
            disc_loss_rnn_val = (criterion_disc(real_outputs_rnn, real_labels) +
                                0.5 * criterion_disc(fake_outputs_rnn, fake_labels) +
                                0.5 * criterion_disc(mismatch_outputs_rnn, fake_labels))
            disc_loss_rnn_val.backward()
            optimizer_disc_rnn.step()
            disc_rnn_losses_main.append(disc_loss_rnn_val.item())

            optimizer_gen.zero_grad()
            encoder_out_gen = encoder(images)
            outputs, _, caps_sorted, lengths_sorted, sort_ind = generator(encoder_out_gen, captions, lengths)

            targets = caps_sorted[:, 1:].reshape(-1)
            outputs_mle = outputs[:, :-1].reshape(-1, outputs.shape[2])
            gen_loss = criterion_gen(outputs_mle, targets)

            encoder_out_gen_sorted_for_scst = encoder_out_gen[sort_ind]

            generated_captions_scst = []
            greedy_captions_scst = []
            for j in range(batch_size):
                sample_scst = generator.sample(encoder_out_gen_sorted_for_scst[j:j+1])
                generated_captions_scst.append(sample_scst)
                with torch.no_grad():
                    greedy_sample_tokens = generator.sample(encoder_out_gen_sorted_for_scst[j:j+1], beam_size=1)
                greedy_captions_scst.append(greedy_sample_tokens)

            sorted_all_captions = [all_captions[k] for k in sort_ind.cpu().numpy()]

            rewards = compute_rewards(
                generated_captions_scst,
                sorted_all_captions,
                ensemble_discriminator,
                encoder_out_gen_sorted_for_scst,
                {"idx2word": idx2word},
                device
            )
            baseline_rewards = compute_rewards(
                greedy_captions_scst,
                sorted_all_captions,
                ensemble_discriminator,
                encoder_out_gen_sorted_for_scst,
                {"idx2word": idx2word},
                device
            )

            adv_caption_lengths_list = [len(ids) if ids else 0 for ids in generated_captions_scst]
            max_len_scst = max(adv_caption_lengths_list) if adv_caption_lengths_list else 0

            adv_captions = torch.zeros(batch_size, max_len_scst, dtype=torch.long, device=device)
            for j, ids in enumerate(generated_captions_scst):
                if ids:
                    adv_captions[j, :len(ids)] = torch.tensor(ids, dtype=torch.long, device=device)

            adv_caption_lengths = torch.tensor(adv_caption_lengths_list, dtype=torch.long, device=device)

            adv_loss = torch.tensor(0.0, device=device)
            if max_len_scst > 1:
                scst_pg_outputs, _, _, _, _ = generator(encoder_out_gen_sorted_for_scst, adv_captions, adv_caption_lengths)

                pg_loss_input_log_probs = F.log_softmax(scst_pg_outputs, dim=2)

                targets_for_pg = adv_captions[:, 1:scst_pg_outputs.size(1) + 1]

                adv_loss = policy_gradient_loss(pg_loss_input_log_probs, rewards, targets_for_pg, baseline_rewards)
            else:
                adv_loss = torch.tensor(0.0, device=device)

            total_gen_loss = lambda_gen * gen_loss + lambda_adv * adv_loss
            total_gen_loss.backward()
            optimizer_gen.step()

            gen_losses.append(gen_loss.item())
            if isinstance(adv_loss, torch.Tensor):
                adv_losses.append(adv_loss.item())

            disc_cnn_losses_main.append(disc_loss_cnn_val.item())
            disc_rnn_losses_main.append(disc_loss_rnn_val.item())

            progress_bar_main.set_postfix({
                'GenLoss': f'{gen_loss.item():.4f}',
                'AdvLoss': f'{adv_loss.item() if isinstance(adv_loss, torch.Tensor) else 0:.4f}',
                'D_CNNLoss': f'{disc_loss_cnn_val.item():.4f}',
                'D_RNNLoss': f'{disc_loss_rnn_val.item():.4f}'
            })
            # Removed 'DEBUG' in os.environ check
        progress_bar_main.close()

        avg_gen_loss = np.mean(gen_losses) if gen_losses else 0
        avg_adv_loss = np.mean(adv_losses) if adv_losses else 0
        avg_disc_cnn_loss = np.mean(disc_cnn_losses_main) if disc_cnn_losses_main else 0
        avg_disc_rnn_loss = np.mean(disc_rnn_losses_main) if disc_rnn_losses_main else 0

        print(f'Epoch {epoch+1}/{num_epochs}:')
        print(f'  Gen Loss: {avg_gen_loss:.4f}')
        print(f'  Adv Loss: {avg_adv_loss:.4f}')
        print(f'  D_CNN Loss: {avg_disc_cnn_loss:.4f}')
        print(f'  D_RNN Loss: {avg_disc_rnn_loss:.4f}')

        # Evaluate on validation set - Removed conditional skip
        val_bleu4, val_meteor, val_rouge_l, val_cider, val_spice = evaluate(val_loader, encoder, generator, word2idx, idx2word, device)

        print(f'  Validation metrics:')
        print(f'    BLEU-4: {val_bleu4:.4f}')
        print(f'    METEOR: {val_meteor:.4f}')
        print(f'    ROUGE-L: {val_rouge_l:.4f}')
        print(f'    CIDEr: {val_cider:.4f}')
        print(f'    SPICE: {val_spice:.4f}')

        torch.save({
            'epoch': epoch,
            'encoder_state_dict': encoder.state_dict(),
            'generator_state_dict': generator.state_dict(),
            'cnn_discriminator_state_dict': cnn_discriminator.state_dict(),
            'rnn_discriminator_state_dict': rnn_discriminator.state_dict(),
            'optimizer_gen_state_dict': optimizer_gen.state_dict(),
            'optimizer_disc_cnn_state_dict': optimizer_disc_cnn.state_dict(),
            'optimizer_disc_rnn_state_dict': optimizer_disc_rnn.state_dict(),
            'gen_loss': avg_gen_loss,
            'adv_loss': avg_adv_loss,
            'disc_cnn_loss': avg_disc_cnn_loss,
            'disc_rnn_loss': avg_disc_rnn_loss,
            'bleu4': val_bleu4,
            'meteor': val_meteor,
            'rouge_l': val_rouge_l,
            'cider': val_cider,
            'spice': val_spice
        }, os.path.join(checkpoint_path, f'checkpoint_epoch_{epoch+1}.pth'))


In [ ]:
from pycocoevalcap.spice.spice import Spice

def evaluate(data_loader, encoder, generator, word2idx, idx2word, device):
    """
    Evaluate model on given data loader with all metrics (original logic with speedup)

    Returns:
        bleu4: BLEU-4 score
        meteor: METEOR score
        rouge_l: ROUGE-L score
        cider: CIDEr-D score
        spice: SPICE score
    """
    encoder.eval()
    generator.eval()

    references = []
    hypotheses = []


    with torch.no_grad():
        for i_batch, (images, _, _, all_captions_batch, image_ids) in enumerate(tqdm(data_loader, desc='Evaluating')):
            images = images.to(device)
            encoder_out = encoder(images)

            for j_img_in_batch in range(images.size(0)):
                sampled_token_ids = generator.sample(encoder_out[j_img_in_batch:j_img_in_batch+1])
                words = [idx2word[token_id] for token_id in sampled_token_ids if token_id not in [0, 1, 2]]
                hypotheses.append(words)

                refs_for_image = all_captions_batch[j_img_in_batch]
                references.append(refs_for_image)

    if not hypotheses or not references:
        print("Warning: No hypotheses or references generated for evaluation. Returning zero scores.")
        return 0.0, 0.0, 0.0, 0.0, 0.0

    smoothie = SmoothingFunction().method4
    bleu4 = corpus_bleu(references, hypotheses, smoothing_function=smoothie, weights=(0.25, 0.25, 0.25, 0.25))

    meteor_scores = []
    for refs_for_one_hyp_tok_lists, hyp_tok_list in zip(references, hypotheses):
        if not hyp_tok_list:
            meteor_scores.append(0.0)
            continue
        valid_refs_present = any(ref_list for ref_list in refs_for_one_hyp_tok_lists)
        if not valid_refs_present:
            meteor_scores.append(0.0)
            continue
        try:
            score = meteor_score(references=refs_for_one_hyp_tok_lists, hypothesis=hyp_tok_list)
            meteor_scores.append(score)
        except Exception as e_meteor:
            print(f"Error during NLTK meteor_score call with tokenized input: {e_meteor}")
            print(f"Hypothesis tokens: {hyp_tok_list}")
            print(f"Reference token lists: {refs_for_one_hyp_tok_lists}")
            meteor_scores.append(0.0)

    meteor = np.mean(meteor_scores) if meteor_scores else 0.0

    rouge = Rouge()
    rouge_scores = []
    for refs_for_one_hyp, hyp_tok_list in zip(references, hypotheses):
        str_hyp = ' '.join(hyp_tok_list)
        if not str_hyp.strip():
            rouge_scores.append(0.0)
            continue
        max_rouge_for_hyp = 0.0
        has_valid_ref = False
        for ref_tok_list in refs_for_one_hyp:
            str_ref = ' '.join(ref_tok_list)
            if not str_ref.strip():
                continue
            has_valid_ref = True
            try:
                score = rouge.get_scores(str_hyp, str_ref)[0]['rouge-l']['f']
                max_rouge_for_hyp = max(max_rouge_for_hyp, score)
            except ValueError:
                pass
        if has_valid_ref:
            rouge_scores.append(max_rouge_for_hyp)
        elif str_hyp.strip():
             rouge_scores.append(0.0)
    rouge_l = np.mean(rouge_scores) if rouge_scores else 0.0

    try:
        cider_scores_list = compute_cider_score(hypotheses, references)
        cider = np.mean(cider_scores_list) if cider_scores_list else 0.0
    except NameError:
        print("Warning: 'compute_cider_score' function not found. CIDEr score will be 0.0. Define it or use direct Cider object.")
        cider = 0.0
    except Exception as e:
        print(f"Error during CIDEr computation: {e}. CIDEr score will be 0.0.")
        cider = 0.0

    spice_scorer = Spice()
    gts_spice = {}
    res_spice = {}
    for i, (refs_for_one_hyp, hyp_tok_list) in enumerate(zip(references, hypotheses)):
        str_refs_spice = [' '.join(ref_tok_list) for ref_tok_list in refs_for_one_hyp if ' '.join(ref_tok_list).strip()]
        str_hyp_spice = ' '.join(hyp_tok_list).strip()

        if not str_refs_spice and not str_hyp_spice:
            continue

        gts_spice[str(i)] = [{'caption': s} for s in str_refs_spice]
        res_spice[str(i)] = [{'caption': str_hyp_spice if str_hyp_spice else ""}]

    if not gts_spice or not res_spice or not any(gts_spice.values()):
        spice = 0.0
    else:
        try:
            common_keys = set(gts_spice.keys()) & set(res_spice.keys())
            gts_spice_filtered = {k: gts_spice[k] for k in common_keys if gts_spice[k]}
            res_spice_filtered = {k: res_spice[k] for k in common_keys}

            if not gts_spice_filtered or not res_spice_filtered:
                spice = 0.0
            else:
                _, spice_scores_eval = spice_scorer.compute_score(gts_spice_filtered, res_spice_filtered)
                if isinstance(spice_scores_eval, list):
                     spice = np.mean([s['All']['f'] for s in spice_scores_eval if isinstance(s, dict) and 'All' in s and 'f' in s['All']]) if spice_scores_eval else 0.0
                elif isinstance(spice_scores_eval, dict) and 'All' in spice_scores_eval and 'f' in spice_scores_eval['All']:
                     spice = spice_scores_eval['All']['f']
                else:
                     spice = 0.0
        except Exception as e:
            print(f"Error during SPICE computation: {e}. SPICE score will be 0.0.")
            spice = 0.0
    return bleu4, meteor, rouge_l, cider, spice

import skimage.transform

def visualize_attention(image_path, caption, alphas, rev_word_map, smooth=True):
    """
    Visualize attention for a given image and caption
    """
    image = Image.open(image_path).convert('RGB')
    image = np.array(image)

    plt.figure(figsize=(15, 15))

    for t in range(len(caption)):
        if t >= len(alphas):
            break

        plt.subplot(np.ceil(len(caption) / 5), 5, t + 1)

        plt.text(0, 1, '%s' % (caption[t]), color='black', backgroundcolor='white', fontsize=12)
        plt.imshow(image)

        alpha = alphas[t]
        if smooth:
            alpha = skimage.transform.pyramid_expand(alpha.cpu().numpy(), upscale=24, sigma=8)
        else:
            alpha = skimage.transform.resize(alpha.cpu().numpy(), [image.shape[0], image.shape[1]])

        plt.imshow(alpha, alpha=0.8, cmap='jet')
        plt.axis('off')

    plt.tight_layout()
    plt.show()

def generate_qualitative_examples(data_loader, encoder, generator, word2idx, idx2word, device, num_examples=5):
    """
    Generate qualitative examples for comparison

    Args:
        data_loader: Validation data loader
        encoder: Image encoder
        generator: Caption generator
        word2idx: Word to index mapping
        idx2word: Index to word mapping
        device: Device (CPU or GPU)
        num_examples: Number of examples to generate
    """
    encoder.eval()
    generator.eval()

    examples = []
    with torch.no_grad():
        for images, _, _, all_captions, image_ids in data_loader:
            images = images.to(device)
            encoder_out = encoder(images)

            for j in range(min(images.size(0), num_examples - len(examples))):
                sample = generator.sample(encoder_out[j:j+1])
                caption = [idx2word[idx] for idx in sample if idx not in [0, 1, 2]]
                ref_captions = [' '.join(ref) for ref in all_captions[j]]
                examples.append({
                    'image_id': image_ids[j],
                    'generated': ' '.join(caption),
                    'references': ref_captions
                })

            if len(examples) >= num_examples:
                break

    for ex in examples:
        print(f"Image ID: {ex['image_id']}")
        print(f"Generated: {ex['generated']}")
        print("References:")
        for ref in ex['references']:
            print(f"  - {ref}")
        print()

In [ ]:
class EnsembleGenerator:
    """
    Ensemble of multiple generator models
    """
    def __init__(self, generators: List[nn.Module]):
        """
        Args:
            generators: List of generator models
        """
        self.generators = generators

    def sample(self, encoder_out: torch.Tensor, beam_size: int = 5) -> List[int]:
        """
        Generate caption by ensembling predictions

        Args:
            encoder_out: Encoded image features
            beam_size: Beam size for beam search

        Returns:
            seq: Generated sequence
        """
        sequences = []
        scores = []

        for generator in self.generators:
            generator.eval()
            seq = generator.sample(encoder_out, beam_size)
            sequences.append(seq)
            # Placeholder score (can use discriminator or CIDEr-D for actual scoring)
            scores.append(1.0 / len(self.generators))

        # Select sequence with highest score (simplified; can use voting or CIDEr-D)
        best_idx = np.argmax(scores)
        return sequences[best_idx]

In [16]:
def generate_caption_for_image(image_path, encoder, generator, word2idx, idx2word, transform=None, beam_size=5):
    """
    Generate caption for a given image
    """
    encoder.eval()
    generator.eval()

    if transform is None:
        transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

    # Load and preprocess image
    image = Image.open(image_path).convert('RGB')
    image = transform(image).unsqueeze(0).to(device)

    # Encode image
    with torch.no_grad():
        if isinstance(encoder, BottomUpEncoder):
            # This case is excluded by the "assuming no bottom-up" condition.
            # If it were to be used, it would need a different logic to load/process features.
            raise NotImplementedError("BottomUpEncoder path is not supported by this fix.")
        elif isinstance(encoder, ImageEncoder):
            encoder_out = encoder(image) # Correct: encoder(image) gives the final features
        else:
            raise TypeError(f"Unsupported encoder type: {type(encoder)}")

    # Generate caption
    # The generator.sample method might expect a specific shape for encoder_out.
    # If encoder_out is (batch_size, feature_dim) and sample expects (batch_size, 1, feature_dim) or similar,
    # you might need to unsqueeze it. Assuming current generator.sample handles (batch_size, feature_dim).
    # For TopDownAttention, sample expects (batch_size, encoder_dim) which matches ImageEncoder output.
    seq = generator.sample(encoder_out, beam_size=beam_size) # Pass beam_size correctly

    # Convert to readable caption
    caption_tokens = []
    for idx_val in seq:
        # If idx_val is a tensor, convert to int
        idx = idx_val.item() if isinstance(idx_val, torch.Tensor) else idx_val
        if idx not in [word2idx['<pad>'], word2idx['<start>'], word2idx['<end>']]: # Use word2idx for special tokens
            caption_tokens.append(idx2word[idx])
    return ' '.join(caption_tokens)

In [17]:
# New Cell: Hyperparameter Tuning

def hyperparameter_tuning(train_loader, val_loader, encoder, generator, cnn_discriminator, rnn_discriminator,
                         word2idx, idx2word, device):
    """
    Perform hyperparameter tuning for lambda, g-steps, and d-steps
    """
    lambda_values = [0.1, 0.3, 0.5, 0.7]
    g_steps = [1, 2]
    d_steps = [1, 2]

    best_bleu4 = 0
    best_params = {}

    for lam, g, d in itertools.product(lambda_values, g_steps, d_steps):
        print(f"Testing lambda={lam}, g_steps={g}, d_steps={d}")

        # Reset models
        encoder.reset_parameters()
        generator.reset_parameters()
        cnn_discriminator.reset_parameters()
        rnn_discriminator.reset_parameters()

        # Train with current hyperparameters
        train_adversarial(
            train_loader, val_loader, encoder, generator, cnn_discriminator, rnn_discriminator,
            word2idx, idx2word, device, learning_rate=3e-4, num_epochs=2,
            lambda_gen=1.0, lambda_adv=lam, checkpoint_path=f'./checkpoints_lambda{lam}_g{g}_d{d}'
        )

        # Evaluate
        bleu4, _, _, _, _ = evaluate(val_loader, encoder, generator, word2idx, idx2word, device)
        print(f"BLEU-4: {bleu4:.4f}")

        if bleu4 > best_bleu4:
            best_bleu4 = bleu4
            best_params = {'lambda': lam, 'g_steps': g, 'd_steps': d}

    print(f"Best parameters: {best_params}, BLEU-4: {best_bleu4:.4f}")
    return best_params

In [ ]:
download_and_extract_coco()

# Set up data paths
train_annotation_file = 'data/coco/annotations/annotations/captions_train2014.json'
val_annotation_file = 'data/coco/annotations/annotations/captions_val2014.json'
image_folder = 'data/coco/images'

# Create vocabulary
word2idx, idx2word = create_vocab_and_tokenize(train_annotation_file)
vocab_size = len(word2idx)
print(f"Vocabulary created with {vocab_size} tokens")

# Define transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
# Create datasets as usual
train_dataset = COCODataset(
    root=image_folder,
    annotation_file=train_annotation_file,
    split='train',
    vocab=word2idx,
    transform=transform,
    use_bottomup=False
)

val_dataset = COCODataset(
    root=image_folder,
    annotation_file=val_annotation_file,
    split='val',
    vocab=word2idx,
    transform=transform,
    use_bottomup=False
)

# Create subset indices - 30% of original size
train_size = int(0.3 * len(train_dataset))
val_size = int(0.3 * len(val_dataset))

# Set seed for reproducibility
random.seed(42)
train_indices = random.sample(range(len(train_dataset)), train_size)
val_indices = random.sample(range(len(val_dataset)), val_size)

# Create samplers for DataLoader
train_sampler = torch.utils.data.SubsetRandomSampler(train_indices)
val_sampler = torch.utils.data.SubsetRandomSampler(val_indices)

# Create data loaders with samplers
train_loader = DataLoader(
    train_dataset,
    batch_size=128,
    sampler=train_sampler,  # Use sampler instead of shuffle
    num_workers=2,
    collate_fn=collate_fn
)

val_loader = DataLoader(
    val_dataset,
    batch_size=128,
    sampler=val_sampler,  # Use sampler instead of shuffle
    num_workers=2,
    collate_fn=collate_fn
)

print(f"Using {train_size} training images (30% of original)")
print(f"Using {val_size} validation images (30% of original)")

# Initialize models
encoder = ImageEncoder(embed_dim=512).to(device)
generator = TopDownAttention(
embed_dim=512,
decoder_dim=1024,
vocab_size=vocab_size,
word2idx=word2idx,  # Pass the word2idx dictionary created earlier
encoder_dim=512
).to(device)

cnn_discriminator = CNNDiscriminator(
    embed_dim=512,
    encoder_dim=512,
    vocab_size=vocab_size,
    hidden_dim=1024
).to(device)

rnn_discriminator = RNNDiscriminator(
    embed_dim=512,
    encoder_dim=512,
    vocab_size=vocab_size,
    hidden_dim=1024
).to(device)

# Training configuration
config = {
    'learning_rate': 3e-4,
    'num_epochs': 1,
    'lambda_gen': 1.0,
    'lambda_adv': 0.2,
    'checkpoint_path': './checkpoints',
    'log_file': './training_log.json'
}

# Initialize training log
training_log = {
    'config': config,
    'epochs': [],
    'best_metrics': {'bleu4': 0, 'meteor': 0, 'rouge_l': 0}
}

# Train with enhanced monitoring
try:
  train_adversarial(
  train_loader=train_loader,
  val_loader=val_loader,
  encoder=encoder,
  generator=generator,
  cnn_discriminator=cnn_discriminator,
  rnn_discriminator=rnn_discriminator,
  word2idx=word2idx,
  idx2word=idx2word,
  device=device,
  learning_rate=5e-3,
  num_epochs=3,
  lambda_gen=1.0,
  lambda_adv=0.1,
  checkpoint_path='./checkpoints')
except KeyboardInterrupt:
  print("\nTraining interrupted. Saving current state...")

# Load best checkpoint
checkpoint_file_path = '/content/checkpoints/checkpoint_epoch.pth' # Assuming this is the correct absolute path
if not os.path.exists(checkpoint_file_path):
    # Fallback to using config path if the absolute one doesn't exist (e.g., running locally)
    checkpoint_file_path = os.path.join(config['checkpoint_path'], 'checkpoint_epoch_1.pth')

print(f"Loading checkpoint from: {checkpoint_file_path}")
best_checkpoint = torch.load(checkpoint_file_path, weights_only=False)
encoder.load_state_dict(best_checkpoint['encoder_state_dict'])
generator.load_state_dict(best_checkpoint['generator_state_dict'])

# Final evaluation
print("\nFinal Evaluation on Validation Set:")
bleu4, meteor, rouge_l, cider, spice = evaluate(val_loader, encoder, generator, word2idx, idx2word, device)

print(f"\nBest Validation Metrics:")
print(f"BLEU-4: {bleu4:.4f}")
print(f"METEOR: {meteor:.4f}")
print(f"ROUGE-L: {rouge_l:.4f}")
print(f"CIDEr: {cider:.4f}")
print(f"SPICE: {spice:.4f}")

# Generate qualitative examples
print("\nSample Validation Captions:")
generate_qualitative_examples(val_loader, encoder, generator, word2idx, idx2word, device, num_examples=5)

# Plot attention visualization
sample_image_path = "/content/data/coco/images/val2014/COCO_val2014_000000000074.jpg"  # Replace with actual path
sample_caption = generate_caption_for_image(sample_image_path, encoder, generator, word2idx, idx2word)

--2025-05-10 22:23:51--  http://images.cocodataset.org/annotations/annotations_trainval2014.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 16.15.192.225, 16.15.200.9, 3.5.27.136, ...
Connecting to images.cocodataset.org (images.cocodataset.org)|16.15.192.225|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 252872794 (241M) [application/zip]
Saving to: ‘data/coco/annotations/annotations_trainval2014.zip’

annotations_trainva 100%[===================>] 241.16M  46.6MB/s    in 5.7s    

2025-05-10 22:23:57 (42.4 MB/s) - ‘data/coco/annotations/annotations_trainval2014.zip’ saved [252872794/252872794]

Extracting annotations...
replace data/coco/annotations/annotations/instances_train2014.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace data/coco/annotations/annotations/instances_val2014.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace data/coco/annotations/annotations/person_keypoints_train2014.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to /root/.cache/torch/hub/checkpoints/resnet101-63fe2227.pth
100%|██████████| 171M/171M [00:00<00:00, 182MB/s]


Pre-training generator with MLE...


Pre-train Gen Epoch 1/25:   0%|          | 0/195 [00:00<?, ?it/s]

Pre-train Gen Epoch 1/25, Gen Loss: 4.2901


...

Epoch 3/3:
  Gen Loss: 2.5700
  Adv Loss: -0.0438
  D_CNN Loss: 0.6872
  D_RNN Loss: 0.0731


Evaluating:   0%|          | 0/95 [00:00<?, ?it/s]

Error during SPICE computation: Command '['java', '-jar', '-Xmx8G', 'spice-1.0.jar', '/usr/local/lib/python3.11/dist-packages/pycocoevalcap/spice/tmp/tmpgx1fxhkl', '-cache', '/usr/local/lib/python3.11/dist-packages/pycocoevalcap/spice/cache', '-out', '/usr/local/lib/python3.11/dist-packages/pycocoevalcap/spice/tmp/tmpxh6h2bzx', '-subset', '-silent']' returned non-zero exit status 1.. SPICE score will be 0.0.
  Validation metrics:
    BLEU-4: 0.2454
    METEOR: 0.4384
    ROUGE-L: 0.4712
    CIDEr: 0.0000
    SPICE: 0.0000
Loading checkpoint from: ./checkpoints/checkpoint_epoch_1.pth

Final Evaluation on Validation Set:


Evaluating:   0%|          | 0/95 [00:00<?, ?it/s]

Error during SPICE computation: Command '['java', '-jar', '-Xmx8G', 'spice-1.0.jar', '/usr/local/lib/python3.11/dist-packages/pycocoevalcap/spice/tmp/tmpvdew__8k', '-cache', '/usr/local/lib/python3.11/dist-packages/pycocoevalcap/spice/cache', '-out', '/usr/local/lib/python3.11/dist-packages/pycocoevalcap/spice/tmp/tmpcrpgkz79', '-subset', '-silent']' returned non-zero exit status 1.. SPICE score will be 0.0.

Best Validation Metrics:
BLEU-4: 0.2449
METEOR: 0.4282
ROUGE-L: 0.4652
CIDEr: 0.0000
SPICE: 0.0000

Sample Validation Captions:
Image ID: 133244
Generated: a man swinging a tennis racket on a tennis court .
References:
  - a man holding a tennis racquet on a tennis court .
  - a man taking a swing at a tennis ball
  - a tennis player is standing close to the net .
  - a man in black shirt and blue shorts playing tennis .
  - a man standing in the middle of a tennis court with a large crowd watching

Image ID: 252701
Generated: a man riding a wave on a surfboard .
References:
  - a

In [20]:
from google.colab import files
import os
import torch

# Save model weights
def save_model_weights(encoder, generator, cnn_discriminator, rnn_discriminator, save_path='/content/checkpoints/final_model_weights.pth'):
    """
    Save the weights of all models to a single file.

    Args:
        encoder: Image encoder model
        generator: Caption generator model
        cnn_discriminator: CNN discriminator model
        rnn_discriminator: RNN discriminator model
        save_path: Path to save the model weights
    """
    os.makedirs(os.path.dirname(save_path), exist_ok=True)
    torch.save({
        'encoder_state_dict': encoder.state_dict(),
        'generator_state_dict': generator.state_dict(),
        'cnn_discriminator_state_dict': cnn_discriminator.state_dict(),
        'rnn_discriminator_state_dict': rnn_discriminator.state_dict(),
    }, save_path)
    print(f"Model weights saved to {save_path}")
    return save_path

# Save the weights
weights_file_path = save_model_weights(encoder, generator, cnn_discriminator, rnn_discriminator)

# Download the weights file to your computer
files.download(weights_file_path)
print(f"Download initiated for {weights_file_path}")

Model weights saved to /content/checkpoints/final_model_weights.pth


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Download initiated for /content/checkpoints/final_model_weights.pth
